# Tide Analysis demonstration for Acapulco tide gauge (IOC)

to make this notebook running, you need: the following packages: 

xarray, pandas, geopandas, ruptures, searvey, utide, cartopy, shapely, matplotlib

See [README.md](https://github.com/tomsail/analysea) for more information
### imports

In [ ]:
from analysea.filters import *
from analysea.utils import detect_gaps, completeness, correct_unit
from analysea.plot import plot_gaps, plot_multiyear_tide_analysis
from analysea.steps import step_function_ruptures
from analysea.spikes import despike_prominence
from analysea.spikes import EWMA 
from analysea.spikes import remove_spikes
from analysea.spikes import buffer_nans
from analysea.tide import yearly_tide_analysis, demean_amps_phases
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
from searvey import ioc
ioc_stations = ioc.get_ioc_stations()

## load test case : Acapulco (Mex)

but you can also try with the other stations in the data folder !

In [ ]:
_ioc_station = "acap2" 
df0 = pd.read_parquet('../tests/data/'+_ioc_station+'.gzip')

In [ ]:
df = pd.DataFrame()
df["slevel"] = remove_numerical(df0.slevel)
df['correct'], units_flag = correct_unit(df.slevel)

# flag the signal
we will start "flagging" the signal to clean it from bad values

## first units
some signals comport different units from the beginning to the end 

example: for ```acap2``` : the beggining in mm and finishing years in m (the norm being in meters)

## then, detect steps

we will use here the Linearly penalized segmentation (Pelt) from the [ruptures](https://centre-borelli.github.io/ruptures-docs/user-guide/detection/pelt/) package
the rest of the processing will be easier once steps will be removed from the signal

In [ ]:
df['step'], stepsx, steps = step_function_ruptures(df.correct)
fig, ax = plt.subplots()
df.correct.plot(ax=ax, color='k')
df.step.plot(ax=ax,color= 'r',linestyle='--')
if (len(stepsx) > 2) & (np.max(steps)>1):
    for i, isteps in enumerate(stepsx[:-1]):
        ax.axvline(x=df.index[isteps], color='k', linestyle='--')
    ipeaks, peaks, df['correct'] = despike_prominence(df.correct - df.step, 1) # despike once already

## spikes
First we will remove the spikes from the signal, defining : 

*NB: it is important to remove the mean after despike https://doi.org/10.3390/rs12233970*

### Method 1: 
Doing the difference between the 'spikey' signal and Forward/Backward exponential weighted moving average (EWMA). 
The difference higher than 3 * standard deviation is removed 

from [here](https://stackoverflow.com/questions/37556487/remove-spikes-from-signal-in-python)

In [ ]:
df['ewma'] = EWMA(df.slevel, 10)
df['despiked'] = remove_spikes(df.slevel, df.ewma, 3*df.slevel.std())
df['clean'] = buffer_nans(df.despiked, 2)

### Method 2 (default): 
Eliminating the absolute values of the signal surpassing a threshold equivalento to 3 time the standard deviation. 
We use Scipy's find_peak() function with a prominence set to ```3*df.std()```

Links & refs: 
 * [Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html)
 * [StackOverFlow](https://stackoverflow.com/questions/1713335/peak-finding-algorithm-for-python-scipy)
 * [Prominence - Wikipedia](https://en.wikipedia.org/wiki/Topographic_prominence)

In [ ]:
prom = np.max([3*df.correct.std(),1])
fig, ax = plt.subplots()
df.correct.plot(ax=ax)
ipeaks, peaks, df['anomaly'] = despike_prominence(df.correct, prom)
ax.plot(df.index[ipeaks], peaks, 'ob')
df.anomaly.plot(ax=ax, color='k')

## Signal to Noise and filtering 
### filter if the signal is noisy

In [ ]:
if signaltonoise(df.slevel)<0: 
    df['filtered'] = filter_fft(df.anomaly)
    
print(signaltonoise(df.anomaly))
print(signaltonoise(df.filtered))

In [ ]:
fig, ax = plt.subplots()
df.anomaly.iloc[2300000:2310000].plot(ax=ax)
df.filtered.iloc[2300000:2310000].plot(ax=ax)

In [ ]:
df.anomaly = df.filtered

## Check continuity of the time series and detect gaps
first some functions


we need to detect gaps either : 
 * interpolate between small gaps (less than a hour)
 * drop data between big gaps (less than a consecutive year)
### First, detect all gaps 

In [ ]:
gaps, small_gaps, big_gaps = detect_gaps(df)
print(f'{len(small_gaps)} small gaps with average gap duration: {small_gaps.mean()}')
print(f'{len(big_gaps)} bigs gaps with average gap duration: {big_gaps.mean()}')
print(f'with the biggest gap being : {big_gaps.max()}')
filenameOutGaps = '../tests/data/graphs/' +  _ioc_station + '_gaps.png'
plot_gaps(df.anomaly,big_gaps,filenameOutGaps)

## check if the time series is worth the tide analysis
if the total time is less than year, we should disregard the time series 

let's define 60 minutes for the minimum interval of no data

In [ ]:
completeness(df.anomaly)

# Detide the signal 

In [ ]:
opts = {'conf_int': 'linear',
        'constit' : 'auto',
        'method' : 'ols', # ols is faster and good for missing data (Ponchaut et al., 2001)
        'order_constit' : 'frequency',
        'Rayleigh_min' : 0.97,
        'lat': None,
        'verbose' : True,
} # careful if there is only one Nan parameter, the analysis crashes

### (de)tide analysis
we analyse the signal by yearly chunks 

In [ ]:
df1 = df.anomaly.reset_index().drop_duplicates(subset="time", keep="last").set_index("time")

In [ ]:
df1

In [ ]:
opts['lat'] = ioc_stations.iloc[np.where(ioc_stations.ioc_code==_ioc_station)[0][0]].lat
df['tide'], df['surge'], coefs, years = yearly_tide_analysis(df1.anomaly, 365, opts)

In [ ]:
# Constituent to keep
keep_const = ['M2', 'S2', 'N2', 'O1', 'K2', 'K1',  'NU2', 'Q1', 'L2', 'P1',
            '2N2', 'M4', 'MS4', 'MM',  'MU2', 'SSA', 'LDA2',  'MF', 'MSM', 'MN4']
ASTRO_WRITE = ["M2","S2","N2","O1","K2","K1","NU2","Q1","L2","P1","2N2","M4","MS4",
               "MM","MU2","SSA","LDA2","MF","MSM","MN"]
lat =  ioc_stations.iloc[np.where(ioc_stations.ioc_code==_ioc_station)[0][0]].lat
lon =  ioc_stations.iloc[np.where(ioc_stations.ioc_code==_ioc_station)[0][0]].lon
title =  ioc_stations.iloc[np.where(ioc_stations.ioc_code==_ioc_station)[0][0]].location
_ioc_code = _ioc_station
filenameOut = '../tests/data/graphs/' + _ioc_code + '.png'

In [ ]:
plot_multiyear_tide_analysis(keep_const, coefs, years, lat, lon, df, title, filenameOut )
plot_multiyear_tide_analysis(keep_const, coefs, years, lat, lon, df, title, filenameOut, zoom=True )

In [ ]:
import json 
const, means_amps,mean_phases = demean_amps_phases(coefs, coefs[0]['name'])
data={
    'lat':lat,
    'lon':lon,
    'const_name': const.tolist(),
    'means_amps': (means_amps).tolist(),
    'mean_phases': mean_phases.tolist(),
    'missing': 100 - completeness(df.anomaly),
    'perc_analysed': len(df.surge) / len(df.slevel),
    'biggest_gap' : str(big_gaps.max()),
    'total_gap' : len(gaps),
    'steps_flag': len(stepsx),
    'unit_flag': units_flag,
    'snr': signaltonoise(df.slevel).tolist(),
    'first_obs' : pd.Timestamp(df.index.min()).strftime('%Y-%m-%d %H:%M:%S'),
    'last_obs' :  pd.Timestamp(df.index.max()).strftime('%Y-%m-%d %H:%M:%S'),
    'code': _ioc_code,
}
resout = pd.DataFrame(data)
with open(f'../tests/data/processed/{_ioc_code}.json', 'w') as f:
    json.dump(data, f)
# resout.to_csv(f'../tests/data/processed/{_ioc_code}.csv', index=False)
# out = pd.DataFrame(data=df.surge, index=df.index)
# out.to_parquet(f'../tests/data/processed/{_ioc_code}_surge.gzip', compression='gzip')

## do the mean on all utide returned coefs

In [ ]:
const, mean_amps,mean_phases = demean_amps_phases(coefs, coefs[0]['name'])

In [ ]:
def json_format(d):
    for key, value in d.items():
        if isinstance(value, dict):
            json_format(value)  # Recurse into nested dictionaries
        elif isinstance(value, np.ndarray):
            d[key] = value.tolist()  # Convert NumPy array to list
        elif isinstance(value, pd.Timestamp):
            d[key] = value.strftime('%Y-%m-%d %H:%M:%S')  # Convert pandas Timestamp to string
        elif isinstance(value, pd.Timedelta):
            d[key] = str(value)  # Convert pandas Timedelta to string
    return d

In [ ]:
js_out = json_format(coefs[-1])
js_out['weights'] = 0 # weights list is too long and unused in the reconstruction
js_out['A'] = mean_amps.tolist()
js_out['g'] = mean_phases.tolist()
# js_out = js_out.pop('weights')
for key in data.keys():
    js_out[key] = data[key]
with open(f"./data/processed/acap2_test.json", "w") as fp:
    json.dump(js_out, fp)

In [ ]:
coef_out = coefs[-1]

In [ ]:
coef_out['A'] = mean_amps
coef_out['g'] = mean_phases

In [ ]:
coef_out